In [6]:
from parse_cpe_xml import *
from registry_query import *
#from similarity import *
from cpe_similarity import *

In [ ]:
software_inventory = get_installed_software()

example_cpe = "cpe:2.3:python:python:3.8.10:*:*:*:*:*:*:*:*"

cpe_vendor = example_cpe.split(":")[2]
cpe_name = example_cpe.split(":")[3]
cpe_version = example_cpe.split(":")[4]

best_match = [{},0]

for s in software_inventory:
    total_similarity = 0

    if s.get("Vendor") and len(s.get("Vendor")) > 0:
        vendor_similarity = string_similarity(cpe_vendor,s.get("Vendor"))
        total_similarity += vendor_similarity

    if s.get("Name") and len(s.get("Name")) > 0:
        name_similarity = string_similarity(cpe_name,s.get("Name"))
        total_similarity += name_similarity
    
    if s.get("Version") and len(s.get("Version")) > 0:
        version_similarity = string_similarity(cpe_version,s.get("Version"))
        total_similarity += version_similarity

    if total_similarity > best_match[1]:
        best_match = [s,total_similarity]

print(best_match)

    

In [2]:
software_inventory = get_installed_software()

In [3]:
cpe_dictionary = return_cpe_df()

In [9]:
all_matches = []

for software in software_inventory:
    best_match = [None,0]

    for cpe in cpe_dictionary.name.to_list():
        if software.get("Vendor") and software.get("Name") and software.get("Version"):
            similarity_score = return_similarity(cpe,software.get("Vendor"),software.get("Name"),software.get("Version"))
        
        else: continue

        if similarity_score > best_match[1] and "a" in cpe.split(":")[1]:
            best_match = [cpe,similarity_score]

    all_matches.append([software,best_match])

for match in all_matches:
    if match[1][0] != None: print(match[1][0])


cpe:/a:amd:amd_link:-::~~~android~~
cpe:/a:daemontools:daemon_tools:3.47.0
cpe:/a:synaptics:fingerprint_driver:5.5.14.1116
cpe:/a:evo:evolution_cms:2.0.4
cpe:/a:samsung:galaxy_watch_plugin:-::~~~android~~
cpe:/a:git:git:2.0.1
cpe:/a:logitech:game_software:9.02.61
cpe:/a:microsoft:365_apps:-
cpe:/a:cisco:unity_express_software:2.2.2
cpe:/a:rarlab:winrar:5.70:beta1
cpe:/a:microsoft:mono:4.3.2.467
cpe:/a:microsoft:visual_c:-
cpe:/a:microsoft:microsoft_tech_companion:1.0.6::~~~android~~
cpe:/a:microsoft:.net:2.0:sp2
cpe:/a:python:python:3.9.7
cpe:/a:microsoft:.net:6.0.0:-
cpe:/a:microsoft:visual_c%2b%2b:8.0
cpe:/a:microsoft:directx:10.1:-
cpe:/a:microsoft:.net:5.0.1
cpe:/a:advanced_reports_project:advanced_reports:1.0::~~~silverstripe~~
cpe:/a:microsoft:.net:5.0.1
cpe:/a:microsoft:.net:6.0.0:-
cpe:/a:microsoft:microsoft_forefront_protection_2010:-::~~~exchange_server~~
cpe:/a:microsoft:word:16.0.8730.2050::~~~android~~
cpe:/a:bigware:bigware_shop:2.0
cpe:/a:microsoft:.net:6.0.0:-
cpe:/a:i-